In [1]:
%pprint

import boto3
import base64
from botocore.exceptions import ClientError

from datetime import datetime

Pretty printing has been turned OFF


In [ ]:
rds_client = boto3.client('rds-data')

In [ ]:
database_name = "data_mart_brand"
db_cluster_arn = "arn:aws:rds:us-east-2:282989224251:cluster:brand-test-2-serverless"
db_credentials_secrets_store_arn = ""

---

### Funções auxiliares para executar comandos

In [ ]:
def execute_statement(sql, sql_parameters=[]):
    response = rds_client.execute_statement(
        secretArn=db_credentials_secrets_store_arn,
        database=database_name,
        resourceArn=db_cluster_arn,
        sql=sql,
        parameters=sql_parameters
    )
    return response

In [ ]:
def batch_execute_statement(sql, sql_parameter_sets):
    response = rds_client.batch_execute_statement(
        secretArn=db_credentials_secrets_store_arn,
        database=database_name,
        resourceArn=db_cluster_arn,
        sql=sql,
        parameterSets=sql_parameter_sets
    )
    return response

---

### Criando tabela

In [ ]:
sql = """
        CREATE TABLE IF NOT EXISTS orders (
            id VARCHAR(50) NOT NULL,
            order_group VARCHAR(50) NOT NULL,
            subaccount VARCHAR(50) NOT NULL,
            seller_order_id VARCHAR(50),
            origin_code INTEGER,
            number_itens INTEGER,
            total_value DOUBLE,
            creation_date DATETIME,
            pickup_itens INTEGER,
            country_code VARCHAR(3),
            currency_code VARCHAR(3),
            is_completed BOOLEAN,
            status VARCHAR(50),
            PRIMARY KEY (id, order_group, subaccount, seller_order_id)
        )
    """

response = execute_statement(sql)

print(response)

### Inserindo dados

#### Funções auxiliares

In [ ]:
sql = """
        INSERT INTO orders 
            (id, order_group, subaccount, seller_order_id, 
                origin_code, number_itens, total_value, 
                creation_date, pickup_itens, country_code, 
                currency_code, is_completed, status) 
        VALUES 
            (:id, :order_group, :subaccount, :seller_order_id, 
                :origin_code, :number_itens, :total_value, 
                :creation_date, :pickup_itens, :country_code, 
                :currency_code, :is_completed, :status) 
        ON DUPLICATE KEY UPDATE 
            is_completed = :is_completed,
            status = :status, 
            number_itens = :number_itens,
            total_value = :total_value,
            pickup_itens = :pickup_itens
    """

def get_sql_params(**kwargs):
    mid = kwargs['id']
    order_group = kwargs['order_group']
    subaccount = kwargs['subaccount']
    seller_order_id = kwargs['seller_order_id']
    origin_code = kwargs['origin_code']
    number_itens = kwargs['number_itens']
    total_value = kwargs['total_value']
    creation_date = kwargs['creation_date']
    pickup_itens = kwargs['pickup_itens']
    seller_order_id = kwargs['seller_order_id']
    country_code = kwargs['country_code']
    currency_code = kwargs['currency_code']
    is_completed = kwargs['is_completed']
    status = kwargs['status']
    
    return [
        {
            'name':'id', 
            'value':{'stringValue': f'{mid}'}
        },
        {
            'name':'order_group', 
            'value':{'stringValue': f'{order_group}'}
        },
        {
            'name':'subaccount', 
            'value':{'stringValue': f'{subaccount}'}
        },
        {
            'name':'seller_order_id', 
            'value':{'stringValue': f'{seller_order_id}'}
        },
        {
            'name':'origin_code', 
            'value':{'stringValue': f'{origin_code}'}
        },
        {
            'name':'number_itens', 
            'value':{'stringValue': f'{number_itens}'}
        },
        {
            'name':'total_value', 
            'value':{'stringValue': f'{total_value}'}
        },
        {
            'name':'creation_date', 
            'value':{'stringValue': f'{creation_date}'}
        },
        {
            'name':'pickup_itens', 
            'value':{'stringValue': f'{pickup_itens}'}
        },
        {
            'name':'country_code', 
            'value':{'stringValue': f'{country_code}'}
        },
        {
            'name':'currency_code', 
            'value':{'stringValue': f'{currency_code}'}
        },
        {
            'name':'is_completed', 
            'value':{'stringValue': f'{is_completed}'}
        },
        {
            'name':'status', 
            'value':{'stringValue': f'{status}'}
        }
    ]

#### Transformando dados para se adequares à tabela do DW

In [2]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
from pyspark.sql.utils import AnalysisException
from pyspark.sql.functions import to_json, struct, col, size, explode, when, UserDefinedFunction
from pyspark.sql.types import Row

spark = SparkSession \
    .builder \
    .appName('Insert orders data on data warehouse')\
    .getOrCreate()

In [3]:
READ_PATH = "../../../data/datalake/raw_data/orders/parquet_stage/ingestion_month=02/ingestion_day=01/ingestion_hour=00"
df = spark.read.parquet(READ_PATH)

In [4]:
cols = [
    'id','hostname','status', 'shippingdata',
    'items','sellerorderid','ordergroup', 'creationdate',
    'origin','value','storepreferencesdata', 'iscompleted'
]
select_df = df.select(*cols)

select_df = select_df.withColumnRenamed("hostname", "subaccount")
select_df = select_df.withColumnRenamed("sellerorderid", "seller_order_id")
select_df = select_df.withColumnRenamed("ordergroup", "order_group")
select_df = select_df.withColumnRenamed("origin", "origin_code")
select_df = select_df.withColumnRenamed("value", "total_value")

iscompleted_to_int = when(col("iscompleted") == True, 1).otherwise(0)
select_df = select_df.withColumn("is_completed", iscompleted_to_int)
select_df = select_df.withColumn("country_code", col("storepreferencesdata.CountryCode"))
select_df = select_df.withColumn("currency_code", col("storepreferencesdata.CurrencyCode"))
select_df = select_df.withColumn("number_itens", size(col("items")))

# from pyspark.sql import Row
# def replace_none_with_null(r):
#     return Row(**{k: None if v == "None" else v for k, v in r.asDict().items()})
# select_df = select_df.rdd.map(lambda x: replace_none_with_null(x)).toDF()

select_df = select_df.drop("items")
select_df = select_df.drop("iscompleted")
select_df = select_df.drop("storepreferencesdata")

# select_df = select_df.limit(50)

In [ ]:
# fixing string creation date

def format_datetime_str(datetime_str):
    datetime_pattern = '%Y-%m-%dT%H:%M:%S.%f'
    datetime_str_without_UTC = datetime_str[:-len('XZ')]
    datetime_object = datetime.strptime(datetime_str_without_UTC, datetime_pattern)
    return str(datetime_object.replace(microsecond=0))

format_datetime_str_UDF = UserDefinedFunction(format_datetime_str, StringType())
select_df = select_df.withColumn('creation_date', format_datetime_str_UDF(select_df.creationdate))
select_df = select_df.drop("creationdate")

In [ ]:
pkeys = [
    'id','subaccount','seller_order_id','order_group'
]

order_item_has_pickup = when(col("PickupStoreInfo.IsPickupStore") == True, 1).otherwise(0)

itmd_df = select_df\
    .select(*pkeys, explode("shippingdata.LogisticsInfo").alias("LogisticsInfo")) \
    .select(*pkeys, "LogisticsInfo.PickupStoreInfo", "LogisticsInfo.ItemIndex") \
    .withColumn("item_with_pickup", order_item_has_pickup) \
    .select(*pkeys, "item_with_pickup", "ItemIndex") \
    .groupby(*pkeys, "item_with_pickup").count()

In [ ]:
# itmd_df.show()

In [ ]:
itmd_df_pickup = itmd_df.where(col("item_with_pickup") == 1)
itmd_df_pickup = itmd_df_pickup.withColumnRenamed("count", "pickup_itens")
itmd_df_pickup = itmd_df_pickup.drop("item_with_pickup")

itmd_df = itmd_df.drop("count", "item_with_pickup")

In [ ]:
# itmd_df_pickup.show()

In [ ]:
joined = itmd_df.join(itmd_df_pickup, on=pkeys, how='left')

In [ ]:
# joined.toPandas().head()

In [ ]:
# Quando order não tem pickup itens, `pickup_itens` = null. Substituindo null por 0.

joined = joined.na.fill(0)

In [ ]:
# Trazendo o restante das colunas para os dados

select_df = select_df.drop('shippingdata')

orders = select_df.join(joined, on=pkeys, how='left')

In [ ]:
# orders.select(*pkeys, 'origin_code').show()

In [ ]:
# Dealing with cases where pickup_itens are null

pickup_null_to_zero = when(col("pickup_itens").isNull(), 0).otherwise(col("pickup_itens"))
orders = orders.withColumn("pickup_itens", pickup_null_to_zero)

In [ ]:
orders.toPandas().to_csv('output.csv', index=False)

#### Enviando dados

In [ ]:
# %%timeit
# transforming df in list
# orders_list = orders.rdd \
#     .map(lambda row: 
#          {'id': row.id, 
#           'subaccount': row.subaccount, 
#           'seller_order_id': row.seller_order_id, 
#           'order_group': row.order_group, 
#           'pickup_itens': row.pickup_itens, 
#           'status': row.status, 
#           'creation_date': row.creation_date, 
#           'origin_code': row.origin_code,
#           'total_value': row.total_value,
#           'is_completed': row.is_completed,
#           'country_code': row.country_code,
#           'currency_code': row.currency_code,
#           'number_itens': row.number_itens}) \
#     .collect()

In [ ]:
# transforming df in list
orders_list = orders.rdd \
    .map(lambda row: 
         {'id': row.id, 
          'subaccount': row.subaccount, 
          'seller_order_id': row.seller_order_id, 
          'order_group': row.order_group, 
          'pickup_itens': row.pickup_itens, 
          'status': row.status, 
          'creation_date': row.creation_date, 
          'origin_code': row.origin_code,
          'total_value': row.total_value,
          'is_completed': row.is_completed,
          'country_code': row.country_code,
          'currency_code': row.currency_code,
          'number_itens': row.number_itens}) \
    .collect()

In [ ]:
# transforming orders list on sql_parameter

sql_parameters_set = [get_sql_params(**row) for row in orders_list]

In [ ]:
%%timeit -n 1 -r 1

# sending batch sql statements in buffers of 100 orders

for i in range(0, len(sql_parameters_set), 100):
    if i+100 > len(sql_parameters_set):
        response = batch_execute_statement(sql, sql_parameters_set[i:])
    else:
        response = batch_execute_statement(sql, sql_parameters_set[i:i+100])

In [ ]:
from pyspark.sql.functions import length
orders.where(length(col("status")) > 20).toPandas()

In [ ]:
orders.head()